<a href="https://colab.research.google.com/github/NtimTim/DS_School_Home_Work/blob/main/Hometask/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_4_Empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/4vel/python-intro/blob/main/Hometask/Task3_dmrf_OOP.ipynb)

**Практика**

Используемые библиотеки


In [131]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
from itertools import groupby
import re

import matplotlib.pyplot as plt
import random
import scipy.stats as sts
%matplotlib inline
from itertools import product

import warnings
warnings.filterwarnings('ignore')

**Загрузка DataFrame**

**Задача 1**

На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.

Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).

In [2]:
# не сработало, TimeoutError
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))


ConnectTimeout: HTTPSConnectionPool(host='data.gov.ru', port=443): Max retries exceeded with url: /opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7a4dd2bf6470>, 'Connection to data.gov.ru timed out. (connect timeout=None)'))

In [125]:
### Если не работает загрузка on-line TimeoutError, а так же в colab так же можно выбрать файл с диска
#from google.colab import files
#uploaded = files.upload() #выберите файл на диске Химический анализ родника в Нескучном саду.csv

df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';', index_col=0)
display(df)

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [128]:
def check_result(result, norma):

    norma = norma.replace(',', '.')
    ntype = norma.split()[:2]
    nnum = norma.split()[2:]
    nnum1 = nnum[0].split('-')[:1]
    nnum2 = nnum[0].split('-')[1:]

    if result == 'б/цвета':
     result = '0'

    if ((ntype[0] == 'в') and (ntype[1] == 'пределах')):
       if ((float(result)>=float(str(nnum1[0])))and(float(result)<=float(str(nnum2[0])))):
         return 'Норма'
       else:
         return 'Отклонение от нормы'
    elif ((ntype[0] == 'не') and (ntype[1] == 'более')):
          if ((float(result)<=float(str(nnum1[0])))):
           return 'Норма'
          else:
           return 'Отклонение от нормы'
    else:
       return 'Обработка результата по нормативу невозможна'

def highlight_value(value, color='red'):
    if value == 'Отклонение от нормы':
        return f'background-color: {color}'
    return

styled_df = df.style.applymap(highlight_value)
styled_df

df['Вывод'] = df.apply(lambda row: check_result(row['Результат анализа'], row['Норматив']), axis=1)
display(df.style.applymap(highlight_value))


,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Норма
Запах,баллы,1,не более 2-3,Норма
Цветность,градусы,б/цвета,не более 30,Норма
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Норма
Нитраты (по NO3),мг/дм3,24,не более 45,Норма
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Норма
Хлориды (Cl),мг/дм3,200,не более 350,Норма


**Теория вероятности. События**

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

**Задача 2**

В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

In [268]:
# решение
# заполняем урну B1
B1 = ['orange']*5 + ['apple']*4
B1
# задаем число экспериментов
#n = 1000   #P_orange ~ 0.162
#n = 10000   #P_orange ~ 0.1711
#n = 100000   #P_orange ~ 0.17012
n = 1000000   #P_orange ~ 0.171427 3 сек
#n = 10000000   #P_orange ~ 0.1716334 29 секунд
# формируем выборку
A = pd.DataFrame({"A":list(map(lambda a:
                                          random.choices(B1, k=3)
                                         , range(n)
                              ))
                 })
# считаем статистику выпадений трёх апельсинов
P_orange = len(A[A.A.apply(lambda x: x == ['orange', 'orange', 'orange'])])/A.shape[0]
# сравниваем результат с 0.119
if P_orange !=  0.119:
  print("В интернете полученный аналитически ответ 0.119 эксперементально не подтверждается, полученное знаение равно", P_orange)
else:
  print("В интернете полученный аналитически ответ 0.119 эксперементально подтверждается, полученное знаение равно", P_orange)

В интернете полученный аналитически ответ 0.119 эксперементально не подтверждается, полученное знаение равно 0.171661


**Задача 3**

Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [276]:
# решение
# как я поняла формулировку "проверит ровно две детали" -
#первая деталь будет нестандартная, а вторая стандартная, т.к. после стандатной мастер уже не проверяет детали

# берем 10 деталей, 3 из которых нестандартные, значит 7 стандартные
B1 =  ['нестандартная']*3 + ['стандартная']*7
B1

# задаем число экспериментов
#n = 1000   #P ~ 0.206
#n = 10000   #P ~ 0.2145
#n = 100000   #P ~ 0.21115
#n = 1000000   #P ~ 0.20975 3 сек
n = 10000000   #P ~ 0.2100539 30 секунд
n = 10000000   #P ~ 0.2100539 30 секунд

# формируем выборку
A = pd.DataFrame({"A":list(map(lambda a:
                                          random.choices(B1, k=2)
                                         , range(n)
                              ))
                 })
# считаем статистику выбора сначала нестандартной детали, потом стандартной детали
P = len(A[A.A.apply(lambda x: x == ['нестандартная', 'стандартная'])])/A.shape[0]
# сравниваем результат с 0.23333
if P !=  0.23333:
  print("В интернете полученный аналитически ответ 0.23333 эксперементально не подтверждается, полученное знаение равно", P)
else:
  print("В интернете полученный аналитически ответ 0.23333 эксперементально подтверждается, полученное знаение равно", P)

В интернете полученный аналитически ответ 0.23333 эксперементально не подтверждается, полученное знаение равно 0.2100539
